# TinyML - GaussianNB (Classifier)

##  1 - Hardcode Implementation

### 1.1 - Importing libraries 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import multivariate_normal
from sklearn.metrics import accuracy_score

### 1.2 - Load Dataset

The Iris dataset is a classic dataset in the field of machine learning and statistics. It was introduced by Sir Ronald A. Fisher in 1936 as an example of discriminant analysis. The dataset is often used for educational purposes and is a common starting point for the practice of pattern classification.


Attributes:

- Sepal length (in centimeters)

- Sepal width (in centimeters)

- Petal length (in centimeters)

- Petal width (in centimeters)

Species:

- 0 - Setosa

- 1 - Versicolor

- 2 - Virginica

In [ ]:
# Load iris dataset
data = load_iris()

# Create a DataFrame
df = pd.DataFrame(data.data, columns=data.feature_names)


# Add target variable to the DataFrame
df['target'] = data.target

# Remove NaN values
df = df.dropna(axis='rows') #remove NaN

# Display the DataFrame
print(df.head())

In [ ]:
classes = np.array(pd.unique(df[df.columns[-1]]), dtype=int)

In [ ]:
classes

In [ ]:
data = df.to_numpy()
nrow,ncol = df.shape
y = data[:,-1]
X = data[:,0:ncol-1]

### 1.3 - Normalization of the data, in order to avoid the effect of the scale of the attributes.

In [ ]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [ ]:
print('Processed data:')
print('Mean: ', np.mean(X, axis = 0))
print('STD:', np.std(X, axis = 0))

### 1.4 - Split into training and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

### 1.5 - Bayesian classifier

Let's consider the parametric case, assuming that each variable is distributed according to a Normal distribution. Other distributions can also be used.

We have already selected the training and test sets. In the training set, we will calculate the mean and standard deviation of each attribute for each class. Next, we classify the data using Bayesian decision theory, i.e.: $X \in C_i$ if, and only if, $P(C_i|X) = \max P(C_j|X)$ for all $j$.



First, we define a function to calculate the joint probability density: $$p(\vec{x}|C_i) = \prod_{j=1}^d p(x_j|C_i), \quad i=1,\ldots, k$$ 
where $C_i$ are the classes. If the distribution is normal, each attribute $X_j$ has the following associated probability density function, for each class:
$$
p(x_j|C_i) = \frac{1}{\sqrt{2\pi\sigma_{C_i}}}\exp \left[ -\frac{1}{2}\left( \frac{x_j-\mu_{C_i}}{\sigma_{C_i}}\right)^2 \right], \quad i=1,2,\ldots, k.
$$
Thus, we have defined a function to calculate the likelihood function.

In [ ]:
from scipy.stats import multivariate_normal

# P - stores the probability of belonging to each class
P = pd.DataFrame(data=np.zeros((X_test.shape[0], len(classes))), columns = classes) 

# Pc - stores the chances of belonging to each class
Pc = np.zeros(len(classes)) #fraction of elements in each class

# For each "i" being a class
for i in np.arange(0, len(classes)):
    # select the elements of class "i"
    elements = tuple(np.where(y_train == classes[i]))
    # Calculates the probability of belonging to class "i"
    Pc[i] = len(elements)/len(y_train)
    # Selects the elements belonging to class "i" in the training set
    Z = X_train[elements,:][0]
    # Calculates the average of the previously selected values
    m = np.mean(Z, axis = 0)
    # Calculates covariance matrix of previously selected elements
    cv = np.cov(np.transpose(Z))
    # Calculates the chance of belonging to each class for the test set
    for j in np.arange(0,X_test.shape[0]):
        x = X_test[j,:]
        # Likelihood function
        pj = multivariate_normal.pdf(x, mean=m, cov=cv, allow_singular=True)
        P[classes[i]][j] = pj*Pc[i]

In [ ]:
print(P)

In [ ]:
y_pred = []
# For each row, calculate the column with the highest probability
for i in np.arange(0, X_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(classes[c])
y_pred = np.array(y_pred, dtype=int)
print(y_pred)

In [ ]:
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

##  2 - Sklearn Implementation

In [ ]:
#!pip install micromlgen  

### 2.1 - Importing libraries

In [2]:
from micromlgen import port
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score

### 2.2 - Load Dataset

The Iris dataset is a classic dataset in the field of machine learning and statistics. It was introduced by Sir Ronald A. Fisher in 1936 as an example of discriminant analysis. The dataset is often used for educational purposes and is a common starting point for the practice of pattern classification.


Attributes:

- Sepal length (in centimeters)

- Sepal width (in centimeters)

- Petal length (in centimeters)

- Petal width (in centimeters)

Species:

- 0 - Setosa

- 1 - Versicolor

- 2 - Virginica

In [3]:
X, y = load_iris(return_X_y=True)

In [4]:
print('Input shape: ', X.shape)
print('Target variable shape: ', y.shape)

Input shape:  (150, 4)
Target variable shape:  (150,)


### 1.3 - Split into training and test data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

### 1.4 - Create the classification model

In [6]:
model = GaussianNB()

### 1.5 - Train the model

In [7]:
model.fit(X_train, y_train)

GaussianNB()

### 1.6 - Evaluating the model with the training data

In [8]:
training_predict = model.predict(X_train)

In [9]:
score = accuracy_score(training_predict, y_train)
print('Accuracy:', score)

Accuracy: 0.9428571428571428


In [10]:
print(metrics.classification_report(y_train, training_predict, digits = 3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        31
           1      0.919     0.919     0.919        37
           2      0.919     0.919     0.919        37

    accuracy                          0.943       105
   macro avg      0.946     0.946     0.946       105
weighted avg      0.943     0.943     0.943       105



In [11]:
print(metrics.confusion_matrix(y_train, training_predict))

[[31  0  0]
 [ 0 34  3]
 [ 0  3 34]]


### 1.7 - Evaluating the model with test data

In [12]:
test_predict = model.predict(X_test)

In [13]:
score = accuracy_score(test_predict, y_test)
print('Accuracy:', score)

Accuracy: 0.9777777777777777


In [14]:
print(metrics.classification_report(y_test, test_predict, digits = 3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        19
           1      1.000     0.923     0.960        13
           2      0.929     1.000     0.963        13

    accuracy                          0.978        45
   macro avg      0.976     0.974     0.974        45
weighted avg      0.979     0.978     0.978        45



In [15]:
print(metrics.confusion_matrix(y_test, test_predict))

[[19  0  0]
 [ 0 12  1]
 [ 0  0 13]]


### 1.8 - Obtaining the model to be implemented in the microcontroller

In [16]:
print(port(model))

#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class GaussianNB {
                public:
                    /**
                    * Predict class for features vector
                    */
                    int predict(float *x) {
                        float votes[3] = { 0.0f };
                        float theta[4] = { 0 };
                        float sigma[4] = { 0 };
                        theta[0] = 4.964516129032; theta[1] = 3.377419354839; theta[2] = 1.464516129032; theta[3] = 0.248387096774;
                        sigma[0] = 0.111966704288; sigma[1] = 0.136586891592; sigma[2] = 0.033257026868; sigma[3] = 0.011529659543;
                        votes[0] = 0.295238095238 - gauss(x, theta, sigma);
                        theta[0] = 5.862162162162; theta[1] = 2.724324324324; theta[2] = 4.210810810811; theta[3] = 1.302702702703;
                        sigma[0] = 0.275325057719; sigma[1] = 0.087246168019; sigm

### 1.9 - Saves the template in a .h file

In [17]:
with open('./GaussianNB/GaussianNB.h', 'w') as file:   
    file.write(port(model, classname='GaussianNB'))
   